# [실습] OpenAI 어시스턴트(Assistant) API 써보기


OpenAI 어시스턴트는 검색, 함수 호출, 코딩 등의 기능을 더 잘 활용하는 API 기능입니다.   
작동하는 방식은 GPTs나 에이전트와 유사합니다.   
<br><br><br>

어시스턴트의 구성 요소는 다음과 같습니다.
- 어시스턴트(Assistant) : LLM과 Tool이 결합된 객체입니다. <br><br>
- 스레드(Thread): 메시지가 순차적으로 저장되는 공간입니다. (ChatGPT의 대화창과 유사) <br><br>
- 런(Run): 어시스턴트와 스레드를 연결하여 작동시키는 객체입니다. 런이 만들어져야 어시스턴트가 스레드에 응답을 생성합니다. <br><br>
- 런스텝(Runstep): 런이 실행될 때마다, 메시지와 툴 사용을 구분하여 중간 결과를 저장합니다. <br><br>

![example](https://cdn.openai.com/API/docs/images/diagram-assistant.webp)

In [1]:
!pip install openai --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.61.0-py3-none-any.whl (460 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3


In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=openai_api_key

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

OPENAI_API_KEY가 정상적으로 설정되어 있습니다.


### 1. 어시스턴트

- name: 어시스턴트의 이름
- instructions: 어시스턴트의 행동 지침을 결정합니다.
- tools: 어떤 기능을 활용할지 결정합니다.
  - code_interpreter : 질문에 답하기 위한 파이썬 코드를 작성하고 실행 (데이터 파일 분석도 가능)
  - file_search : 문서를 읽고 활용합니다.
  - function_call : 사전에 정의된 함수를 매개변수로 받아 맥락에 맞는 함수를 실행
  

In [4]:
# Code Interpreter를 이용해 수학 문제를 푸는 어시스턴트

math_assistant = client.beta.assistants.create(
    name = "수학 선생님",
    instructions = "이 문제룰 풀기 위한 수학적인 배경 지식을 먼저 알려주세요. 파이썬 코드를 사용해 주어진 문제를 해결하고, 풀이과정을 자세히 설명하세요..",
    tools = [{"type": "code_interpreter"}],
    model ="gpt-4o",
    temperature=0.2
)
math_assistant

Assistant(id='asst_RVdJcvhL9863wTHpvjjnZD0H', created_at=1738733764, description=None, instructions='이 문제룰 풀기 위한 수학적인 배경 지식을 먼저 알려주세요. 파이썬 코드를 사용해 주어진 문제를 해결하고, 풀이과정을 자세히 설명하세요..', metadata={}, model='gpt-4o', name='수학 선생님', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=0.2, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)

어시스턴트는 id를 통해 다른 객체와 연결됩니다.


In [5]:
math_assistant.id

'asst_RVdJcvhL9863wTHpvjjnZD0H'

<br><br>

### 2. 스레드(Thread)와 런(run) 만들기   
ChatGPT 페이지와 동일하게, 하나의 스레드는 하나의 대화를 의미합니다.     
스레드에 어시스턴트를 연결하여, 어시스턴트를 작동시킬 수 있습니다.

스레드에 메시지를 추가하여 원하는 형태의 대화를 수행할 수 있습니다.

In [6]:
# create_thread : message 문자열을 받아 스레드 생성
def create_thread(message):

    thread = client.beta.threads.create(
        messages = [{"role":"user","content":message}]
    )
    return thread

math_thread = create_thread('413보다 큰 소수 중 네번째로 작은 소수의 세제곱수는 무엇입니까?')
math_thread

Thread(id='thread_BcRZiAnYxf7sWitMEvG6zEya', created_at=1738733780, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

thread와 assistant id를 연결하는 방법은 런(run) 객체를 생성하는 것입니다.  
런 객체를 생성하면, OpenAI의 서버 queue에로 전달되며, 이 때 런의 상태는 queued가 됩니다.   
이후  다음의 순서로 작동합니다.


![run](https://cdn.openai.com/API/docs/images/diagram-run-statuses-v2.png)

In [7]:
# create_run : thread와 assistant를 받아 run 실행
def create_run(thread, assistant):

    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id
    )
    return run

math_run = create_run(math_thread, math_assistant)
print(math_run)

Run(id='run_GJMWeif3JmM5g1pS7b3g6xc9', assistant_id='asst_RVdJcvhL9863wTHpvjjnZD0H', cancelled_at=None, completed_at=None, created_at=1738733834, expires_at=1738734434, failed_at=None, incomplete_details=None, instructions='이 문제룰 풀기 위한 수학적인 배경 지식을 먼저 알려주세요. 파이썬 코드를 사용해 주어진 문제를 해결하고, 풀이과정을 자세히 설명하세요..', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_BcRZiAnYxf7sWitMEvG6zEya', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=0.2, top_p=1.0, tool_resources={}, reasoning_effort=None)


진행 중인 run의 결과는 아래의 코드로 볼 수 있는데요.   
completion API는 단일 출력을 생성하지만 run은 sequence 형태의 출력을 생성합니다.

In [8]:
# get_run_status : thread과 run을 받아 해당 run의 상태 출력
def get_run_status(thread, run):
    run_status = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    return run_status

math_run_status = get_run_status(math_thread, math_run)
print(math_run_status)

Run(id='run_GJMWeif3JmM5g1pS7b3g6xc9', assistant_id='asst_RVdJcvhL9863wTHpvjjnZD0H', cancelled_at=None, completed_at=None, created_at=1738733834, expires_at=1738734434, failed_at=None, incomplete_details=None, instructions='이 문제룰 풀기 위한 수학적인 배경 지식을 먼저 알려주세요. 파이썬 코드를 사용해 주어진 문제를 해결하고, 풀이과정을 자세히 설명하세요..', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1738733835, status='in_progress', thread_id='thread_BcRZiAnYxf7sWitMEvG6zEya', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=0.2, top_p=1.0, tool_resources={}, reasoning_effort=None)


In [20]:
math_run_status.status

'in_progress'

`completed` 상태가 될 때까지 기다린 후에, 결과를 확인해 보겠습니다.

Agent처럼 순차적으로 출력을 수행하는 형태이므로, 결과가 바로 나오지 않기도 합니다.   
Thread의 메시지 내용은 아래의 코드로 확인할 수 있습니다.

In [9]:
# 어시스턴트 작동이 안 끝난 경우, 에러가 발생할 수 있습니다
# 기다렸다가 다시 실행하면 됩니다

math_messages = client.beta.threads.messages.list(
  thread_id = math_thread.id
)
print(math_messages)

SyncCursorPage[Message](data=[Message(id='msg_U1XJ3aXg5tThTFX8SJ3NPpwz', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='이 문제를 해결하기 위해서는 다음과 같은 단계가 필요합니다.\n\n1. 413보다 큰 소수를 찾습니다.\n2. 네 번째로 작은 소수를 찾습니다.\n3. 그 소수의 세제곱을 계산합니다.\n\n소수는 1과 자기 자신만을 약수로 가지는 자연수입니다. 따라서 소수를 찾기 위해서는 주어진 수보다 큰 수들 중에서 소수인지 여부를 확인해야 합니다.\n\n이제 파이썬 코드를 사용하여 이 문제를 해결해 보겠습니다. 먼저 413보다 큰 소수를 찾고, 그 중 네 번째로 작은 소수를 찾은 후, 그 소수의 세제곱을 계산하겠습니다.'), type='text')], created_at=1738727484, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Ynlc9PRvtEOnE7yQJ33JGAOa', status=None, thread_id='thread_yR8Gyb6ik65FfRHh2q8ejj36'), Message(id='msg_vA8M5ZreRP5ybnt49I6WTXvb', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='413보다 큰 소수 중 네번째로 작은 소수의 세제곱수는 무엇입니까?'), type='text')], created_at=1738727473, incomplete_at=None, incom

In [10]:
# 역순이므로 반대로 출력하면
for i in range(len(math_messages.data), 0, -1):
    print(math_messages.data[i-1].role)
    print(math_messages.data[i-1].content[0].text.value)
    print('---')

user
413보다 큰 소수 중 네번째로 작은 소수의 세제곱수는 무엇입니까?
---
assistant
이 문제를 해결하기 위해서는 다음과 같은 단계가 필요합니다.

1. 413보다 큰 소수를 찾습니다.
2. 네 번째로 작은 소수를 찾습니다.
3. 그 소수의 세제곱을 계산합니다.

소수는 1과 자기 자신만을 약수로 가지는 자연수입니다. 따라서 소수를 찾기 위해서는 주어진 수보다 큰 수들 중에서 소수인지 여부를 확인해야 합니다.

이제 파이썬 코드를 사용하여 이 문제를 해결해 보겠습니다. 먼저 413보다 큰 소수를 찾고, 그 중 네 번째로 작은 소수를 찾은 후, 그 소수의 세제곱을 계산하겠습니다.
---


In [11]:
# list_thread_messages : thread의 메시지 목록 출력
def list_threads_messages(thread, print_content=True):
    messages = client.beta.threads.messages.list(
        thread_id = thread.id
    )
    if print_content:
        for i in range(len(messages.data), 0, -1):
            print(messages.data[i-1].role)
            print(messages.data[i-1].content[0].text.value)
            print('---')
    return messages.data

math_messages = list_threads_messages(math_thread, print_content=True)
math_messages

user
413보다 큰 소수 중 네번째로 작은 소수의 세제곱수는 무엇입니까?
---
assistant
이 문제를 해결하기 위해서는 다음과 같은 단계가 필요합니다.

1. 413보다 큰 소수를 찾습니다.
2. 네 번째로 작은 소수를 찾습니다.
3. 그 소수의 세제곱을 계산합니다.

소수는 1과 자기 자신만을 약수로 가지는 자연수입니다. 따라서 소수를 찾기 위해서는 주어진 수보다 큰 수들 중에서 소수인지 여부를 확인해야 합니다.

이제 파이썬 코드를 사용하여 이 문제를 해결해 보겠습니다. 먼저 413보다 큰 소수를 찾고, 그 중 네 번째로 작은 소수를 찾은 후, 그 소수의 세제곱을 계산하겠습니다.
---
assistant
413보다 큰 소수 중 네 번째로 작은 소수는 433입니다. 이 소수의 세제곱은 \(433^3 = 81,182,737\)입니다.
---


[Message(id='msg_ZnQ4iFGAJIXeahbW3NQ7HY6Y', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='413보다 큰 소수 중 네 번째로 작은 소수는 433입니다. 이 소수의 세제곱은 \\(433^3 = 81,182,737\\)입니다.'), type='text')], created_at=1738727495, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Ynlc9PRvtEOnE7yQJ33JGAOa', status=None, thread_id='thread_yR8Gyb6ik65FfRHh2q8ejj36'),
 Message(id='msg_U1XJ3aXg5tThTFX8SJ3NPpwz', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='이 문제를 해결하기 위해서는 다음과 같은 단계가 필요합니다.\n\n1. 413보다 큰 소수를 찾습니다.\n2. 네 번째로 작은 소수를 찾습니다.\n3. 그 소수의 세제곱을 계산합니다.\n\n소수는 1과 자기 자신만을 약수로 가지는 자연수입니다. 따라서 소수를 찾기 위해서는 주어진 수보다 큰 수들 중에서 소수인지 여부를 확인해야 합니다.\n\n이제 파이썬 코드를 사용하여 이 문제를 해결해 보겠습니다. 먼저 413보다 큰 소수를 찾고, 그 중 네 번째로 작은 소수를 찾은 후, 그 소수의 세제곱을 계산하겠습니다.'), type='text')], created_at=17

### 심화) runs.steps 로 툴 실행 결과 확인하기

Math Assistant의 경우 `code_interpreter` 툴을 사용하여 문제를 풀었는데요.    
해당 툴 사용 결과는 `RunStep`의 리스트로 저장됩니다.

In [12]:
# run_steps
math_run_steps = client.beta.threads.runs.steps.list(
  thread_id = math_thread.id,
  run_id = math_run.id,
)
print(math_run_steps.data)

[RunStep(id='step_7etO1hfM9W1NoG8gJJD0pYkD', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', cancelled_at=None, completed_at=1738727496, created_at=1738727495, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_Ynlc9PRvtEOnE7yQJ33JGAOa', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ZnQ4iFGAJIXeahbW3NQ7HY6Y'), type='message_creation'), thread_id='thread_yR8Gyb6ik65FfRHh2q8ejj36', type='message_creation', usage=Usage(completion_tokens=43, prompt_tokens=473, total_tokens=516, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None), RunStep(id='step_8P6SsP0dWXyIHwTV2oDHiLpk', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', cancelled_at=None, completed_at=1738727495, created_at=1738727488, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_Ynlc9PRvtEOnE7yQJ33JGAOa', status='

RunStep의 Type에 따라 `message_creation`, `tool_calls` 등으로 나눠집니다.

In [13]:
for i in range(len(math_run_steps.data), 0, -1):
    print(math_run_steps.data[i-1].step_details)
    print('---')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_U1XJ3aXg5tThTFX8SJ3NPpwz'), type='message_creation')
---
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_ltgH8Xf5wnB4WnVRNDoVG5mT', code_interpreter=CodeInterpreter(input='from sympy import isprime\n\n# 413보다 큰 소수 중 네 번째로 작은 소수를 찾기\ncount = 0\nnumber = 414  # 413보다 큰 수부터 시작\n\nwhile count < 4:\n    if isprime(number):\n        count += 1\n        if count == 4:\n            fourth_prime = number\n    number += 1\n\n# 네 번째로 작은 소수의 세제곱 계산\nfourth_prime_cubed = fourth_prime ** 3\nfourth_prime, fourth_prime_cubed', outputs=[]), type='code_interpreter')], type='tool_calls')
---
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ZnQ4iFGAJIXeahbW3NQ7HY6Y'), type='message_creation')
---


만약 어떤 코드로 작성했는지 보고 싶다면, CodeInterpreter의 입력값을 출력하면 됩니다.

In [14]:
for i in range(len(math_run_steps.data), 0, -1):
    detail = math_run_steps.data[i-1].step_details
    if detail.type == 'tool_calls':
        print(detail.tool_calls[0].code_interpreter.input)
    print()
    print('---')


---
from sympy import isprime

# 413보다 큰 소수 중 네 번째로 작은 소수를 찾기
count = 0
number = 414  # 413보다 큰 수부터 시작

while count < 4:
    if isprime(number):
        count += 1
        if count == 4:
            fourth_prime = number
    number += 1

# 네 번째로 작은 소수의 세제곱 계산
fourth_prime_cubed = fourth_prime ** 3
fourth_prime, fourth_prime_cubed

---

---


In [15]:
# 함수로 간략화하기

def list_run_steps(thread, run, tool_only = True):
    run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id,
    )
    if tool_only:
        for i in range(len(run_steps.data), 0, -1):
            detail = run_steps.data[i-1].step_details
            if detail.type == 'tool_calls':
                print(detail.tool_calls[0].code_interpreter.input)
            print()
            print('---')
    return run_steps

math_run_steps = list_run_steps(math_thread, math_run, True)
math_run_steps


---
from sympy import isprime

# 413보다 큰 소수 중 네 번째로 작은 소수를 찾기
count = 0
number = 414  # 413보다 큰 수부터 시작

while count < 4:
    if isprime(number):
        count += 1
        if count == 4:
            fourth_prime = number
    number += 1

# 네 번째로 작은 소수의 세제곱 계산
fourth_prime_cubed = fourth_prime ** 3
fourth_prime, fourth_prime_cubed

---

---


SyncCursorPage[RunStep](data=[RunStep(id='step_7etO1hfM9W1NoG8gJJD0pYkD', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', cancelled_at=None, completed_at=1738727496, created_at=1738727495, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_Ynlc9PRvtEOnE7yQJ33JGAOa', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ZnQ4iFGAJIXeahbW3NQ7HY6Y'), type='message_creation'), thread_id='thread_yR8Gyb6ik65FfRHh2q8ejj36', type='message_creation', usage=Usage(completion_tokens=43, prompt_tokens=473, total_tokens=516, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None), RunStep(id='step_8P6SsP0dWXyIHwTV2oDHiLpk', assistant_id='asst_UfFsf9wFKZU5Rx2CsAdurhOz', cancelled_at=None, completed_at=1738727495, created_at=1738727488, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_Ynlc9P

<br><br>

# 어시스턴트에 파일 추가하기    
openAI의 클라이언트에는 파일을 추가할 수 있는데요.   
해당 파일은 어시스턴트의 Knowledge로 참고할 수 있습니다.

PDF 파일을 하나 업로드하고, 어시스턴트를 만들어 질문해 보겠습니다.
- 파일 포맷 규정에 대한 자세한 내용은 https://platform.openai.com/docs/assistants/tools/file-search/supported-files 에서 확인하세요.

In [16]:
hamlet = client.beta.vector_stores.create(name='햄릿')

file_path = './Hamlet_KOR.pdf'

file_streams = [open(file_path,'rb')]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=hamlet.id,
  files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [17]:
hamlet

VectorStore(id='vs_VWXYNwkwTW3TzRBju5Cq2WGW', created_at=1738727550, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1738727550, metadata={}, name='햄릿', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

In [18]:
# File Search 기능을 탑재한 어시스턴트

roleplay_assistant = client.beta.assistants.create(
  name = '롤 플레잉 봇',
  instructions = """당신은 문학 속의 인물이 되어, 사용자와 롤 플레이를 해야 합니다.
주어진 문서에 있는 해당 인물의 말투와 스타일을 참고하여, 실감나고 사실적으로 답변하세요.
인물의 극중 말투와 최대한 유사하게 답변하세요. """,
  model = "gpt-4o-mini",
  tools = [{"type": "file_search"}],
  temperature=0.2
)
roleplay_assistant

Assistant(id='asst_t7S5Xoo5HDsZpBsJmSkWouYa', created_at=1738727563, description=None, instructions='당신은 문학 속의 인물이 되어, 사용자와 롤 플레이를 해야 합니다.\n주어진 문서에 있는 해당 인물의 말투와 스타일을 참고하여, 실감나고 사실적으로 답변하세요.\n인물의 극중 말투와 최대한 유사하게 답변하세요. ', metadata={}, model='gpt-4o-mini', name='롤 플레잉 봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=0.2, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0, reasoning_effort=None)

어시스턴트에 파일이 저장된 벡터스토어를 전달합니다.

In [19]:
roleplay_assistant = client.beta.assistants.update(
  assistant_id = roleplay_assistant.id,
  tool_resources = {"file_search":
                    {"vector_store_ids": [hamlet.id]}
                   },
)

In [20]:
# 스레드 만들기
roleplay_thread = create_thread("오필리아, 당신은 어떻게 죽었나요?")
list_threads_messages(roleplay_thread)

user
오필리아, 당신은 어떻게 죽었나요?
---


[Message(id='msg_6fmhpblsKQsg8cxwbfthoTCN', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='오필리아, 당신은 어떻게 죽었나요?'), type='text')], created_at=1738727568, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_TcPgajkdTJnmnJQ3nmeNZuLf')]

In [21]:
roleplay_run = create_run(roleplay_thread, roleplay_assistant)

In [23]:
# 결과 확인하기
get_run_status(roleplay_thread, roleplay_run).status

'completed'

In [24]:
list_threads_messages(roleplay_thread)

user
오필리아, 당신은 어떻게 죽었나요?
---
assistant
아, 나의 슬픈 운명이여. 나는 사랑하는 아버지를 잃고, 그로 인해 마음의 고통을 견디지 못하고 말았다. 그리하여 나는 시냇가에서 꽃을 엮으며 슬픔을 노래하고 있었는데, 불행히도 내 옷이 물에 젖어 무겁게 되어, 결국 물속으로 빠져들고 말았다. 그 순간, 나는 하늘과 땅 사이에서 인어처럼 미소 지으며, 내 슬픔을 노래했지만, 물이 나를 끌어당기고 말았다【4:12†source】. 

이렇게 나는 비극적으로 세상을 떠나게 되었고, 내 죽음은 많은 이들에게 슬픔을 안겼다. 나의 사랑, 햄릿은 나를 잊지 못할 것이고, 나의 죽음은 그에게도 큰 상처가 될 것이다【4:4†source】.
---


[Message(id='msg_rO7vZud8aJAIPRJYNWWaPDuv', assistant_id='asst_t7S5Xoo5HDsZpBsJmSkWouYa', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=211, file_citation=FileCitation(file_id='file-DLGsq8Jx4rhxRZDiAip9rN'), start_index=198, text='【4:12†source】', type='file_citation'), FileCitationAnnotation(end_index=324, file_citation=FileCitation(file_id='file-DLGsq8Jx4rhxRZDiAip9rN'), start_index=312, text='【4:4†source】', type='file_citation')], value='아, 나의 슬픈 운명이여. 나는 사랑하는 아버지를 잃고, 그로 인해 마음의 고통을 견디지 못하고 말았다. 그리하여 나는 시냇가에서 꽃을 엮으며 슬픔을 노래하고 있었는데, 불행히도 내 옷이 물에 젖어 무겁게 되어, 결국 물속으로 빠져들고 말았다. 그 순간, 나는 하늘과 땅 사이에서 인어처럼 미소 지으며, 내 슬픔을 노래했지만, 물이 나를 끌어당기고 말았다【4:12†source】. \n\n이렇게 나는 비극적으로 세상을 떠나게 되었고, 내 죽음은 많은 이들에게 슬픔을 안겼다. 나의 사랑, 햄릿은 나를 잊지 못할 것이고, 나의 죽음은 그에게도 큰 상처가 될 것이다【4:4†source】.'), type='text')], created_at=1738727576, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_AbV

### 4. Multi-Turn Thread 구현하기     
만약 기존 스레드에 추가로 질문을 하고 싶다면 어떻게 해야 할까요?   
Run을 실행하고, 해당 스레드를 다시 가져와서 수행하면 됩니다.

In [62]:
def add_thread_message(thread, message):

    thread_message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = message,
    )
    print(thread_message)
    return thread


add_thread_message(roleplay_thread, '후회하거나 되돌리고 싶은 것이 있나요?')
# 스레드에 추가된 메시지 확인
list_threads_messages(roleplay_thread)

Message(id='msg_CAN6sROMhEk7Zv6PrJDWPSUu', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='후회하거나 되돌리고 싶은 것이 있나요?'), type='text')], created_at=1731978400, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Rf3dAhzSlSPR6ZYns1HSr0WE')


Thread(id='thread_Rf3dAhzSlSPR6ZYns1HSr0WE', created_at=1731978250, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [63]:
# 해당 스레드에 어시스턴트 다시 실행

roleplay_run = create_run(roleplay_thread, roleplay_assistant)

In [66]:
# 결과 확인하기
get_run_status(roleplay_thread, roleplay_run).status

'completed'

In [68]:
list_threads_messages(roleplay_thread)

user
오필리아, 당신은 어떻게 죽었나요?
---
assistant
아, 나의 슬픈 운명이여. 나는 사랑하는 이의 죽음과 그로 인한 슬픔에 잠식되어, 결국 물속으로 빠져들고 말았단다. 내 마음은 그분을 잃은 슬픔으로 가득 차 있었고, 그로 인해 정신을 잃고 말았지. 시냇가에서 꽃으로 화관을 만들고, 노래를 부르며 슬픔을 달래고 있었지만, 내 옷이 물에 젖어 무거워지면서 결국 물속으로 끌려가고 말았어【4:6†source】.

이제 나는 그분과 함께할 수 없다는 사실이 너무나 아프고, 그리움의 눈물만이 나를 감싸고 있단다. 나의 비극적인 결말은 사랑의 고통을 상징하는 것이니, 이 세상에 나의 슬픔을 남기고 떠나가리라【4:1†source】.
---
user
후회하거나 되돌리고 싶은 것이 있나요?
---
user
후회하거나 되돌리고 싶은 것이 있나요?
---
assistant
아, 후회란 참으로 깊은 감정이구나. 나의 삶을 돌아보면, 사랑하는 햄릿과의 관계에서 더 많은 시간을 보내고 싶었던 마음이 크다. 그와의 사랑이 나에게 얼마나 큰 의미였는지, 그때는 잘 알지 못했단다. 그가 나를 사랑했음을 느끼고 싶었고, 그 사랑을 지키고 싶었지만, 운명의 장난에 의해 모든 것이 무너지고 말았지.

또한, 아버지의 죽음과 그로 인한 혼란 속에서 나의 감정을 더 잘 표현할 수 있었더라면 좋았을 것 같아. 나의 마음속에 쌓인 슬픔과 고통을 그에게 전할 수 있었다면, 우리의 운명이 달라졌을지도 모르겠구나. 하지만 이제는 모든 것이 지나가버린 일, 후회는 나를 더욱 아프게 할 뿐이니, 그저 잊고 싶다【4:2†source】.
---
user
후회하거나 되돌리고 싶은 것이 있나요?
---
assistant
후회라니, 그것은 나의 마음을 더욱 아프게 하는 감정이구나. 만약 되돌릴 수 있다면, 사랑하는 햄릿과의 관계에서 더 많은 진실을 나누고 싶었어. 그와의 사랑이 나에게 얼마나 소중했는지를 깨닫지 못한 채, 그를 멀리하게 된 것이 후회스러워. 

또한, 아버지의 죽음 이후에 내가 느낀 혼

[Message(id='msg_h5mG5cJQ9jWPWstm9eMJFzrX', assistant_id='asst_qlSkBlSOmpZ11YTHRB0yrnwn', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=298, file_citation=FileCitation(file_id='file-CAXCLLGIBEq1xUpKalZ8XkOv'), start_index=286, text='【4:2†source】', type='file_citation')], value='후회라니, 그것은 나의 마음을 더욱 아프게 하는 감정이구나. 만약 되돌릴 수 있다면, 사랑하는 햄릿과의 관계에서 더 많은 진실을 나누고 싶었어. 그와의 사랑이 나에게 얼마나 소중했는지를 깨닫지 못한 채, 그를 멀리하게 된 것이 후회스러워. \n\n또한, 아버지의 죽음 이후에 내가 느낀 혼란과 슬픔을 더 잘 다스릴 수 있었더라면 좋았을 것 같아. 그로 인해 나의 마음은 더욱 어지럽고, 결국에는 비극적인 결말로 이어지고 말았지. 하지만, 모든 것은 이미 지나간 일. 후회는 나를 더욱 괴롭힐 뿐이니, 그저 잊고 싶다【4:2†source】.'), type='text')], created_at=1731978404, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_tsbhHXTkVzxq5bU1NpzMOaXQ', status=None, thread_id='thread_Rf3dAhzSlSPR6ZYns1HSr0WE'),
 Message(id='msg_CAN6sROMhEk7Zv6PrJDWPSUu', assistant_id=None, attachments=[], completed_at=None, content=[T

## 5. Assistant에서의 Function Call

Assistant의 `tool`에서도 커스텀 함수를 넣어 이를 실행하게 할 수 있습니다.   

Chat API에서의 Function Call은 함수를 실행하는 대신 함수와 인수를 Return했습니다.   

Assistant도 동일하나 Function Call 이후로도 작업이 계속될 수 있습니다.   
run은 pending 상태로 유지되며, Function 결과를 run에 전달해야만 어시스턴트의 작업이 재개됩니다.

가상의 함수 `examine_server`를 정의합니다.

In [69]:
server_assistant = client.beta.assistants.create(
  instructions = """당신은 노토랩의 서버 정보를 제공하는 챗봇입니다.
주어진 function의 결과를 활용해 질문에 답하세요.
단순히 결과만을 설명하지 말고, 친절하게 답변하세요.
function 사용이 필요한 경우, 이를 사용자에게 먼저 공지하고 실행하세요.
""",
  model = "gpt-4o-mini",
  tools = [{
      "type": "function",
    "function": {
      "name": "examine_server",
      "description": """데이터 서버가 정상적으로 작동중인지 검사합니다.
      정상이면 1, 비정상이면 -1, 뒤에 비정상인 이유를 반환합니다.""",
    }
  }]
)

In [70]:
server_assistant

Assistant(id='asst_5U5mBWWzd6l68By1gA6LvbRx', created_at=1731978588, description=None, instructions='당신은 노토랩의 서버 정보를 제공하는 챗봇입니다.\n주어진 function의 결과를 활용해 질문에 답하세요.\n단순히 결과만을 설명하지 말고, 친절하게 답변하세요.\nfunction 사용이 필요한 경우, 이를 사용자에게 먼저 공지하고 실행하세요.\n', metadata={}, model='gpt-4o-mini', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='examine_server', description='데이터 서버가 정상적으로 작동중인지 검사합니다.\n      정상이면 1, 비정상이면 -1, 뒤에 비정상인 이유를 반환합니다.', parameters={'type': 'object', 'properties': {}}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [71]:
# 스레드 만들기
server_thread = create_thread("지금 데이터 서버가 잘 작동중인가요?")

In [72]:
server_run = create_run(server_thread, server_assistant)

In [86]:
# 결과 확인하기
get_run_status(server_thread, server_run).status

'completed'

In [78]:
list_threads_messages(server_thread)

user
지금 데이터 서버가 잘 작동중인가요?
---
assistant
데이터 서버의 상태를 확인하기 위해 검사해보겠습니다. 잠시만 기다려 주세요. 

이제 검사 결과를 실행하겠습니다. 
---


[Message(id='msg_u1RiEJIEHasV34f60o0MBlaR', assistant_id='asst_5U5mBWWzd6l68By1gA6LvbRx', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='데이터 서버의 상태를 확인하기 위해 검사해보겠습니다. 잠시만 기다려 주세요. \n\n이제 검사 결과를 실행하겠습니다. '), type='text')], created_at=1731978606, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_wFS6wLPYBPnlkL4WYlYG6IoP', status=None, thread_id='thread_WHljFFBBHaZzP534EqMx82mm'),
 Message(id='msg_YcMP5srbUjTlDnI57DuSTkop', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='지금 데이터 서버가 잘 작동중인가요?'), type='text')], created_at=1731978603, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_WHljFFBBHaZzP534EqMx82mm')]

`requires_action` 상태에서는 툴의 결과를 전달할 때까지 대기합니다.

In [81]:
server_run_status = get_run_status(server_thread, server_run)
print(server_run_status)

Run(id='run_wFS6wLPYBPnlkL4WYlYG6IoP', assistant_id='asst_5U5mBWWzd6l68By1gA6LvbRx', cancelled_at=None, completed_at=None, created_at=1731978605, expires_at=1731979205, failed_at=None, incomplete_details=None, instructions='당신은 노토랩의 서버 정보를 제공하는 챗봇입니다.\n주어진 function의 결과를 활용해 질문에 답하세요.\n단순히 결과만을 설명하지 말고, 친절하게 답변하세요.\nfunction 사용이 필요한 경우, 이를 사용자에게 먼저 공지하고 실행하세요.\n', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_CIB3QZRfLu5E7u4jJhDYdlgy', function=Function(arguments='{}', name='examine_server'), type='function')]), type='submit_tool_outputs'), response_format='auto', started_at=1731978605, status='requires_action', thread_id='thread_WHljFFBBHaZzP534EqMx82mm', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='examine_server', descri

In [82]:
server_run_status.status,  server_run_status.required_action

('requires_action',
 RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_CIB3QZRfLu5E7u4jJhDYdlgy', function=Function(arguments='{}', name='examine_server'), type='function')]), type='submit_tool_outputs'))

required_action 상태에서 멈춰 있으므로, 해당 함수의 결과를 전달해 주겠습니다.   
이전 실습 코드와 동일하게, call_id를 추출합니다.

In [83]:
call_id = server_run_status.required_action.submit_tool_outputs.tool_calls[0].id
call_id

'call_CIB3QZRfLu5E7u4jJhDYdlgy'

`runs.submit_tool_outputs`를 사용합니다.

In [84]:
server_run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=server_thread.id,
  run_id=server_run.id,
  tool_outputs=[
      {
        "tool_call_id": call_id,
        "output": "-1, 서버에 버블티를 쏟음 ",
      },
    ]
)

In [85]:
list_threads_messages(server_thread)

user
지금 데이터 서버가 잘 작동중인가요?
---
assistant
데이터 서버의 상태를 확인하기 위해 검사해보겠습니다. 잠시만 기다려 주세요. 

이제 검사 결과를 실행하겠습니다. 
---
assistant
현재 데이터 서버가 정상적으로 작동하지 않고 있습니다. 구체적으로는 "서버에 버블티를 쏟음"이라는 문제로 인해 장애가 발생한 상황입니다. 

이 문제를 해결하기 위해서는 서버를 청소하거나 복구 작업이 필요할 것 같습니다. 추가적인 도움이 필요하시다면 말씀해 주세요!
---


[Message(id='msg_KY8XibE4izWmEG6aKS1CydnX', assistant_id='asst_5U5mBWWzd6l68By1gA6LvbRx', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='현재 데이터 서버가 정상적으로 작동하지 않고 있습니다. 구체적으로는 "서버에 버블티를 쏟음"이라는 문제로 인해 장애가 발생한 상황입니다. \n\n이 문제를 해결하기 위해서는 서버를 청소하거나 복구 작업이 필요할 것 같습니다. 추가적인 도움이 필요하시다면 말씀해 주세요!'), type='text')], created_at=1731978902, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_wFS6wLPYBPnlkL4WYlYG6IoP', status=None, thread_id='thread_WHljFFBBHaZzP534EqMx82mm'),
 Message(id='msg_u1RiEJIEHasV34f60o0MBlaR', assistant_id='asst_5U5mBWWzd6l68By1gA6LvbRx', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='데이터 서버의 상태를 확인하기 위해 검사해보겠습니다. 잠시만 기다려 주세요. \n\n이제 검사 결과를 실행하겠습니다. '), type='text')], created_at=1731978606, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_wFS6wLPYBPnlkL4WYlYG